# Tesla assignment :
"""
Task:
 
Please write an application in any language that calls the USGS API and store the result in a relational database of your choice.
 
https://earthquake.usgs.gov/fdsnws/event/1/
 
1) Please query all events that have occurred during year 2017

2) Read a JSON response from the API

3) Design the database objects required to store the result in a relational fashion.

4) Store the response in those objects

5) Provide query/analysis to give biggest earthquake of 2017

6) Provide query/analysis to give most probable hour of the day for the earthquakes
bucketed by the range of magnitude (0-1,1-2,2-3,3-4,4-5,5-6,>6,
For border values in the bucket, include them in the bucket where the value is a lower limit
so for 1 include it in 1-2 bucket)
 


For question 1-4 provide following:

Code for the integration

DB schema

For question 5 and 6 provide following:

Queries for the analysis

Any interesting visualization (using any open source product or trial version of any product) for these questions. Just attach screenshot.
"""

In [1]:
# import libs
import mysql.connector
import requests
import pandas as pd
import time
import seaborn as sns

/Users/ankitbhardwaj/opt/miniconda3/envs/assignment-tesla-aug-2022/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Create request urls for each month.
urls = {}
months = ['01','02','03','04','05','06','07','08','09','10','11']
for month in months:
    start_time = f'2017-{month}-01'
    end_time = f'2017-{str(int(month)+1)}-01'
    date_range = f'starttime=2014-01-01&endtime=2014-01-02'
    urls[f"{start_time} to {end_time}"] = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_time}&endtime={end_time}"

urls[f"2017-12-01 to 2018-01-01"] = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2017-12-01&endtime=2018-01-01"

print(str(urls))

{'2017-01-01 to 2017-2-01': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2017-01-01&endtime=2017-2-01', '2017-02-01 to 2017-3-01': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2017-02-01&endtime=2017-3-01', '2017-03-01 to 2017-4-01': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2017-03-01&endtime=2017-4-01', '2017-04-01 to 2017-5-01': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2017-04-01&endtime=2017-5-01', '2017-05-01 to 2017-6-01': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2017-05-01&endtime=2017-6-01', '2017-06-01 to 2017-7-01': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2017-06-01&endtime=2017-7-01', '2017-07-01 to 2017-8-01': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2017-07-01&endtime=2017-8-01', '2017-08-01 to 2017-9-01': 'https://earthquake.usgs.gov/fdsnws/event

In [3]:
# set up RDBMS with mySql.

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password=""
)
_DATABASE = "Tesla"
_TABLE = "earthquakes"

cursor = mydb.cursor()
_CREATE_DATABASE = f'''CREATE DATABASE IF NOT EXISTS {_DATABASE}'''
_SELECT_DATABASE = f'''USE {_DATABASE}'''
_COLUMNS = "(event_id, place, mag, time, longitude, latitude, depth)"
_CREATE_TABLE = """CREATE TABLE  IF NOT EXISTS earthquakes ( 
                             id int(11) NOT NULL AUTO_INCREMENT,
                             event_id VARCHAR(20) NOT NULL,
                             place VARCHAR(100),
                             mag FLOAT,
                             time bigint(20) NOT NULL,
                             longitude FLOAT,
                             latitude FLOAT,
                             depth FLOAT,
                             PRIMARY KEY (Id)) """
cursor.execute(_CREATE_DATABASE)
cursor.execute(_SELECT_DATABASE)
cursor.execute(_CREATE_TABLE)


In [7]:
# Send a GET request for each month and 
payload = {}
for date_range, url in urls.items():
    sql = f"INSERT INTO {_TABLE} {_COLUMNS} VALUES (%s, %s, %s, %s, %s, %s, %s)"
    print(f"fetching the data for {date_range}")
    payload = requests.get(url).json()
    val = []
    for feature in payload['features']:
        event_id = feature['id']
        mag = feature['properties']['mag']
        time = feature['properties']['time']
        place = feature['properties']['place']
        coordinates = feature['geometry']['coordinates']
        longitude = coordinates[0]
        latitude = coordinates[1]
        depth = coordinates[2]

        val.append((event_id, place, mag, time, longitude, latitude, depth))

    print(f"loading the data for {date_range}")
    # cursor.executemany(sql, val)
    mydb.commit()

    

fetching the data for 2017-01-01 to 2017-2-01
loading the data for 2017-01-01 to 2017-2-01
fetching the data for 2017-02-01 to 2017-3-01
loading the data for 2017-02-01 to 2017-3-01
fetching the data for 2017-03-01 to 2017-4-01
loading the data for 2017-03-01 to 2017-4-01
fetching the data for 2017-04-01 to 2017-5-01
loading the data for 2017-04-01 to 2017-5-01
fetching the data for 2017-05-01 to 2017-6-01
loading the data for 2017-05-01 to 2017-6-01
fetching the data for 2017-06-01 to 2017-7-01
loading the data for 2017-06-01 to 2017-7-01
fetching the data for 2017-07-01 to 2017-8-01
loading the data for 2017-07-01 to 2017-8-01
fetching the data for 2017-08-01 to 2017-9-01
loading the data for 2017-08-01 to 2017-9-01
fetching the data for 2017-09-01 to 2017-10-01
loading the data for 2017-09-01 to 2017-10-01
fetching the data for 2017-10-01 to 2017-11-01
loading the data for 2017-10-01 to 2017-11-01
fetching the data for 2017-11-01 to 2017-12-01
loading the data for 2017-11-01 to 2017

### ANALYSIS

In [5]:
# load back the data from Mysql ( could be a decoupled job )
cursor.execute('SELECT * FROM earthquakes')
table_rows = cursor.fetchall()
df = pd.DataFrame(table_rows, columns=cursor.column_names).set_index('Id')

#  transformstions for the requested data.
filtered_df = df[df.mag.notnull()][df['mag'] >= 0]
filtered_df["hour"] = [time.gmtime(datetime).tm_hour for datetime in filtered_df["time"]]
filtered_df["rounded_magnitude"] = [f'{int(mag)} - {int(mag)+1}' if mag < 6 else ">6" for mag in filtered_df["mag"]]


# query 5:
strongest_earthquake = filtered_df.query('mag == mag.max()').iloc[0]
print("strongest_earthquake")
print(strongest_earthquake)


# query 6:
agg = filtered_df.groupby(['rounded_magnitude', 'hour'])['time'].count().reset_index(name="count")

agg['rank'] = agg.groupby(['rounded_magnitude'])['count'].rank(method='first', ascending=False)
hour_for_earthquakes = agg[agg['rank'] == 1.0]
print("highest probable hour of earth quakes bucketed by the range requested of magnitude")
hour_for_earthquakes.head(10)
cm = sns.light_palette("lightgreen", as_cmap=True)
agg.sort_values(by=['rounded_magnitude', 'count']).style.background_gradient(cmap=cm,subset=pd.IndexSlice[:, ['hour']])


/var/folders/bd/1t8hs32d3l723nhyn31sdddw0000gn/T/ipykernel_31238/3458996756.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_df = df[df.mag.notnull()][df['mag'] >= 0]


event_id                                    us2000ahv0
place                near the coast of Chiapas, Mexico
mag                                                8.2
time                                     1504846159180
longitude                                     -93.8993
latitude                                       15.0222
depth                                            47.39
hour                                                21
rounded_magnitude                                   >6
Name: 101515, dtype: object


,rounded_magnitude,hour,count,rank
3,0 - 1,3,3914,1.000000
24,1 - 2,0,4566,1.000000
70,2 - 3,22,1466,1.000000
82,3 - 4,10,368,1.000000
111,4 - 5,15,1028,1.000000
132,5 - 6,12,158,1.000000
158,>6,14,18,1.000000


In [11]:
agg.sort_values(by=['rounded_magnitude', 'hour']).style.background_gradient(cmap=cm,subset=pd.IndexSlice[:, ['rank']])

,rounded_magnitude,hour,count,rank
0,0 - 1,0,3852,5.000000
1,0 - 1,1,3672,19.000000
2,0 - 1,2,3748,15.000000
3,0 - 1,3,3914,1.000000
4,0 - 1,4,3864,4.000000
5,0 - 1,5,3740,17.000000
6,0 - 1,6,3654,21.000000
7,0 - 1,7,3774,10.000000
8,0 - 1,8,3872,3.000000
9,0 - 1,9,3750,14.000000
